After observing the yahoo finance package, found some days' closing price are not correct. So, the extra script help to fix the data. There is no need to download and fix manully everytime. I already email to the yahoo finance to fix this data. If they fix, then there will be no use of this first script in future.


In [1]:
import yfinance as yf
import pandas as pd

def download_and_fix_eurusd_close():
    # Download EUR/USD data
    print("Downloading EUR/USD data...")
    ticker = "EURUSD=X"
    df = yf.download(ticker, start="2000-01-01", progress=False)

    df = df[['Close']].copy()

    # Flatten columns if multi-index (common in yfinance)
    if isinstance(df.columns, pd.MultiIndex):
        print("Flattening columns...")
        df.columns = df.columns.get_level_values(0)

    # Taken from https://www.kaggle.com/datasets/lehenzehra/eurusd-daily-data-ohlc?select=EURUSD_D1_Sorted.csv
    corrections = {
        "2008-01-08": 1.4705,
        "2008-02-08": 1.4503,
        "2008-08-08": 1.5074,
        "2008-09-08": 1.4250,
        "2008-10-08": 1.3650,
        "2008-12-08": 1.2930,
    }

    print("\nApplying corrections...")

    # Apply corrections
    for date_str, price in corrections.items():
        dt = pd.Timestamp(date_str)
        if dt in df.index:
            print(f"Fixing {date_str}: Old={df.at[dt, 'Close']:.4f} -> New={price:.4f}")
            df.at[dt, 'Close'] = price
        else:
            print(f"Warning: {date_str} not found in data.")


    # Save to CSV
    output_file = "EURUSD_Close_Fixed.csv"
    df.to_csv(output_file)
    print(f"\nDone. Saved to {output_file}")


download_and_fix_eurusd_close()

/tmp/ipykernel_18199/2381738963.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2000-01-01", progress=False)


Flattening columns...

Applying corrections...
Fixing 2008-01-08: Old=1.5571 -> New=1.4705
Fixing 2008-02-08: Old=1.5571 -> New=1.4503
Fixing 2008-08-08: Old=1.5049 -> New=1.5074
Fixing 2008-09-08: Old=1.5050 -> New=1.4250
Fixing 2008-10-08: Old=1.4957 -> New=1.3650
Fixing 2008-12-08: Old=1.4918 -> New=1.2930

Done. Saved to EURUSD_Close_Fixed.csv


Load the corrected data and make a 'difference' percentafge column based on the closing price.

In [2]:
df = pd.read_csv("EURUSD_Close_Fixed.csv", index_col=0, parse_dates=True)
df['difference'] = df['Close'].pct_change() * 100
df.head()
df.to_csv('EURUSD_Close_Fixed_with_difference.csv')


The main script started from here. Creating main data table for calculating Euclidean Distane.

In [3]:
data = df['difference'].dropna().tolist()

main_data_rows = []
window_size = 6

# Iterate through the data to create feature vectors and target values
for i in range(len(data) - window_size + 1):
    row = data[i : i + window_size]
    feature_vector = row[:5]
    true_value = row[5]
    main_data_rows.append(feature_vector + [true_value])

# Create column names for the new DataFrame
column_names = [f'p{j+1}' for j in range(5)] + ['true_value_next_day']

# Create the 'main_data' DataFrame
main_data = pd.DataFrame(main_data_rows, columns=column_names)

# Set the DataFrame index to start from 1
main_data.index = range(1, len(main_data) + 1)

#Saving "Main Data Table"
main_data.to_csv('main_data_table.csv', index_label='Vectors')
print("Saved 'main_data_table.csv'.")

Saved 'main_data_table.csv'.


After forming the "difference" column as vectors, calcultaing the Euclidean Distance. And do statics of the positive and negative outcomes.

In [5]:
import numpy as np
from datetime import datetime

#Create base_vector range -2 to 2, each time take 5 datapoints for a vectors.
generated_base_vectors = np.random.uniform(low=-2.0, high=2.0, size=(200, 5))


all_results = []
start_time = datetime.now()
print(f"Started at => {start_time}")
print(f"\n-----Calculating Euclidean Distance-----\n")
for current_threshold in np.arange(0.5, 1.51, 0.1):
    print(f"T = {current_threshold:.1f};")

    # Iterate through each generated base vector
    for gen_base_vec_idx in range(generated_base_vectors.shape[0]):
        current_base_np_vector = generated_base_vectors[gen_base_vec_idx]
        accepted_forecasts_for_this_base_vector = []
        euclidean_distance = 0

        for index, row in main_data.iterrows():
            feature_vector = np.array(row[['p1', 'p2', 'p3', 'p4', 'p5']].tolist())
            euclidean_distance = np.linalg.norm(feature_vector - current_base_np_vector)

            if euclidean_distance < current_threshold:
                accepted_forecasts_for_this_base_vector.append(row['true_value_next_day'])

        accepted_series_for_this_base_vector = pd.Series(accepted_forecasts_for_this_base_vector)

        positive_outcomes = (accepted_series_for_this_base_vector > 0).sum()
        negative_outcomes = (accepted_series_for_this_base_vector <= 0).sum()
        total_accepted_forecasts = len(accepted_series_for_this_base_vector)

        all_results.append({
          'THRESHOLD': f"{current_threshold:.1f}",
          'base_vector': current_base_np_vector,
          'ED': euclidean_distance,
          'accepted_forecast': accepted_forecasts_for_this_base_vector,
          'Total Accepted Forecasts': total_accepted_forecasts,
          'Positive Outcomes': f'{(positive_outcomes/total_accepted_forecasts)*100:.2f}' if total_accepted_forecasts > 0 else 0,
          'Negative Outcomes': f'{(negative_outcomes/total_accepted_forecasts)*100:.2f}' if total_accepted_forecasts > 0 else 0,
      })

result_df = pd.DataFrame(all_results)
print("\nResults Different THRESHOLDS:")
display(result_df)
result_df.to_csv('overall_reuslt.csv', index= True)
end_time = datetime.now()
print(f"Finished at => {end_time}")
print(f"Total time take => {end_time - start_time}")


Started at => 2025-12-21 23:32:12.855184

-----Calculating Euclidean Distance-----

T = 0.5;
T = 0.6;
T = 0.7;
T = 0.8;
T = 0.9;
T = 1.0;
T = 1.1;
T = 1.2;
T = 1.3;
T = 1.4;
T = 1.5;

Results Different THRESHOLDS:


,THRESHOLD,base_vector,ED,accepted_forecast,Total Accepted Forecasts,Positive Outcomes,Negative Outcomes
0,0.5,"[1.867620285351487, -1.4986967862763514, 0.782...",2.239955,[],0,0,0
1,0.5,"[-0.21136389507647513, -0.5506689473731128, -0...",1.975680,[-0.45905801493235643],1,0.00,100.00
2,0.5,"[0.8572622542256036, -1.9970926311239698, 0.10...",2.575492,[],0,0,0
3,0.5,"[-0.48010444555458465, -0.9548751494560292, 0....",2.038403,"[-0.9385695753211465, -0.7589664336749347]",2,0.00,100.00
4,0.5,"[0.7448058049659645, -1.7514264925484708, 1.55...",3.170502,[],0,0,0
...,...,...,...,...,...,...,...
2195,1.5,"[-1.4553546178394856, -0.8309740912780801, -0....",2.506327,"[-0.8611017601265702, 1.0503023372326004, 0.71...",107,46.73,53.27
2196,1.5,"[-1.2578908700351925, -0.8536797415361721, -1....",2.783563,"[-1.4271490427547606, 0.4918644245357484, 0.36...",45,46.67,53.33
2197,1.5,"[-1.973064266252977, -1.230692266877012, -0.54...",2.754289,"[1.0503023372326004, -0.8487108222996365, 0.24...",49,46.94,53.06
2198,1.5,"[-1.3885403933439902, -1.4024235642750233, -1....",2.880912,"[-0.8487108222996365, 0.4918644245357484, -1.7...",30,36.67,63.33


Finished at => 2025-12-22 01:20:00.025855
Total time take => 1:47:47.170671
